In [1]:
# 📘 02_llama_finetuning.ipynb

import pandas as pd

# Load synthetic Q&A data
qa_path = "../data/fiqa_dataset/fiqa_synthetic_qa.csv"
df_qa = pd.read_csv(qa_path)

print("✅ Sample Q&A pair:")
print(df_qa.sample(3))


✅ Sample Q&A pair:
                                              question  \
316  The Intelligent Investor: Northern Pacific Rai...   
960  Can a single-member LLC have a fiscal year not...   
49       Buying car from rental business without title   

                                                answer  
316  Here's the 2009-2014 return of the S&P 500 (SP...  
960  Amazon has 2 different cards you can apply for...  
49   "According to an article on Bankrate.com from ...  


In [2]:
# Format each Q&A pair into instruction-prompt format
def format_example(row):
    return f"### User:\n{row['question']}\n\n### Assistant:\n{row['answer']}"

# Apply formatting
df_qa["formatted"] = df_qa.apply(format_example, axis=1)

# Preview formatted examples
print("✅ Formatted sample:")
print(df_qa["formatted"].sample(1).values[0])

# Save to file for training
df_qa["formatted"].to_csv("../data/fiqa_dataset/fiqa_llama_formatted.txt", index=False, header=False)


✅ Formatted sample:
### User:
Is a website/domain name an asset or a liability?

### Assistant:
I always find this funny. How can government bonds be in attractive and currency be attractive?  With monetary policy America guarantees that it can't default on debt. The only thing that can happen which breaks this is if the government prints itself out of debt. In which case not only will you bonds be worthless but so will your cash.  So to all the investors with boats of cash, you are trading one problem for the same problem. The only difference is you can hold the second problem in your hands.  Fools.


In [ ]:
from huggingface_hub import login
login(token="YOUR_ACCESS_TOKEN")

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "openlm-research/open_llama_3b"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set a folder to store temporarily offloaded weights
offload_dir = "./offload_weights"
os.makedirs(offload_dir, exist_ok=True)

# Load model with offloading support
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    offload_folder=offload_dir
)


Some parameters are on the meta device because they were offloaded to the cpu and disk.


Error while downloading from https://cdn-lfs.hf.co/repos/8c/6e/8c6e9be084f67ff0b804baac0bd770fce63cb974ff15f95ee53080dbfd72aadf/f8da6a367550184167fcb64570699c2997fd4c8602a8c29e82d47e82a1880938?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1745186051&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTE4NjA1MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy84Yy82ZS84YzZlOWJlMDg0ZjY3ZmYwYjgwNGJhYWMwYmQ3NzBmY2U2M2NiOTc0ZmYxNWY5NWVlNTMwODBkYmZkNzJhYWRmL2Y4ZGE2YTM2NzU1MDE4NDE2N2ZjYjY0NTcwNjk5YzI5OTdmZDRjODYwMmE4YzI5ZTgyZDQ3ZTgyYTE4ODA5Mzg%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=joTGrvxb98OI7wkRoS9s-WOsQQRz31xVOZ2984TYDdyEvTRlNf95cLmxO4EsoheoLICHxcUpPBWi6GfDo-Hn7Ad-pEFCz%7E3jYGnq2itvddnsxfp48k15%7E%7EhCC-pvYtP9-fq7S5bNIvKVXyq8qDAAAO8NXRpZC09Qmy1WZ2hdm7FyKbozYM6qE1iTfsH2F3NQ9FKb26AQe-30wgcGpaMLsyF-NYB0bag0FXknqJ9i5MwEmdFa9%7E6b8mM1pteGgeIL34E

In [13]:
from peft import LoraConfig, get_peft_model, TaskType
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 2,662,400 || all params: 3,429,136,000 || trainable%: 0.0776
